NLP Project Phase 3 - Modeling

In [99]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [100]:
#loading the datasets 
#as masking datasets have more positive outcomes in my case, I would be using it for model training
mask_data1 = pd.read_csv("/home/akshay/Desktop/IIT_courses/NLP/NLP_Phase3/change_org_masking.csv")
mask_data2 = pd.read_csv("/home/akshay/Desktop/IIT_courses/NLP/NLP_Phase3/twitter_masking.csv")


In [101]:
mask_data1.head()

,text,label
0,Language Education in the Time of COVID-19,False
1,COVID-19 Test Kits,False
2,COVID 19 IN PRISON,False
3,Get Waled Home,False
4,Make pass/fail available for Mississippi State...,False


In [102]:
mask_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 2 columns):
text     1500 non-null object
label    1500 non-null bool
dtypes: bool(1), object(1)
memory usage: 13.3+ KB


In [103]:
mask_data2.tail()

,text,label
1195,.Sprint To Develop A #COVIDVaccine – // https:...,False
1196,Here in the U.S. some localities have brought ...,True
1197,Sanitizer &amp; Mask Manufacturers After Russi...,False
1198,We are following all CDC guidelines through a ...,True
1199,People be losing hopes on covid vaccine like i...,False


In [104]:
mask_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
text     1200 non-null object
label    1200 non-null object
dtypes: object(2)
memory usage: 18.8+ KB


In [105]:
#preparing data 
#here, we can see that 3 unwanted columns in the dataset. So, we will concatenate the data from them and drop the columns
#print(mask_data2.loc[2,"label"])
#for i in range(0,1200):
#    if(mask_data2.loc[p,"label"] != True or mask_data2.loc[p,"label"] != False):
#        print("need changes")


In [106]:
#concatenating both the dataframes
#mask_data = pd.concat([mask_data1, mask_data2])
#print(mask_data)

In [107]:
#mask_data.iloc[2699]


In [108]:
#resetting index of the dataframe 
#mask_data = mask_data.reset_index(drop = True)

In [109]:
#mask_data.tail()

In [110]:
#1] splitting the primary dataset into train and test sets
x_train, x_test1, y_train, y_test1 = train_test_split(mask_data1["text"],mask_data1["label"], test_size=0.3, random_state=42 )

In [111]:
#print(x_train["text"])

In [112]:
#using full secodnary dataset as test dataset
x_test2 = mask_data2["text"]
y_test2 = mask_data2["label"] == 'True'
#y_test2.dtype
#print(y_test2)

In [113]:
print(x_train.dtype)
print(y_train.dtype)
print(x_test1.dtype)
print(y_test1.dtype)
print(x_test2.dtype)
print(y_test2.dtype)

object
bool
object
bool
object
bool


In [114]:
#print(x_test2)

In [115]:
print(len(x_train))
print(len(x_test1))
print(len(y_train))
print(len(y_test1))
print(len(x_test2))
print(len(y_test2))

1050
450
1050
450
1200
1200


In [116]:
#2] Baseline Modeling - Bag of words
from sklearn.feature_extraction.text import CountVectorizer

In [117]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(x_train)
x_train_counts.shape

(1050, 2984)

In [118]:
count_vect.vocabulary_.get(u'algorithm')

In [119]:
#tf-idf
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
x_train_tfidf = tfidf.fit_transform(x_train_counts)
x_train_tfidf.shape

(1050, 2984)

In [120]:
#Training a naive bayes classifier on training dataset
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB().fit(x_train_tfidf, y_train)


In [121]:
x_test1_counts = count_vect.transform(x_test1)
x_test1_tfidf = tfidf.transform(x_test1_counts)

pred = nb.predict(x_test1_tfidf)

In [122]:
#for txt, category in zip(x_test1, pred):
#    print('%r => %s' % (txt, y_test1[category]))

In [123]:
#performance evaluation with primary test dataset

accu_primary = np.mean(pred == y_test1)
#print(pred)
#print(y_test1)
print("Accuracy of primary test dataset is")
print(accu_primary)

Accuracy of primary test dataset is
0.7555555555555555


In [124]:
from sklearn.metrics import classification_report

print(classification_report(y_test1, pred))

              precision    recall  f1-score   support

       False       0.74      1.00      0.85       313
        True       1.00      0.20      0.33       137

   micro avg       0.76      0.76      0.76       450
   macro avg       0.87      0.60      0.59       450
weighted avg       0.82      0.76      0.69       450



In [125]:
#performance evaluation with sceondary dataset

x_test2_counts = count_vect.transform(x_test2)
x_test2_tfidf = tfidf.transform(x_test2_counts)

pred2 = nb.predict(x_test2_tfidf)
#print(pred2)
#print(y_test2)
accu_secondary = np.mean(pred2 == y_test2)
print("Accuracy of secondary test dataset is")
print(accu_secondary)

Accuracy of secondary test dataset is
0.7825


In [126]:
from sklearn.metrics import classification_report

print(classification_report(y_test2, pred2))

              precision    recall  f1-score   support

       False       0.78      1.00      0.88       939
        True       0.00      0.00      0.00       261

   micro avg       0.78      0.78      0.78      1200
   macro avg       0.39      0.50      0.44      1200
weighted avg       0.61      0.78      0.69      1200



/home/akshay/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [127]:
#using SVM model on training data
from sklearn.linear_model import SGDClassifier

svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None).fit(x_train_tfidf, y_train)

/home/akshay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [128]:
#predicting using SVM model for primary dataset 
pred = svm.predict(x_test1_tfidf)

In [129]:
#performance evaluation with primary test dataset

accu_primary = np.mean(pred == y_test1)
#print(pred)
#print(y_test1)
print("Accuracy of primary test dataset is")
print(accu_primary)

Accuracy of primary test dataset is
0.8177777777777778


In [130]:
#performance evaluation with sceondary dataset

pred2 = svm.predict(x_test2_tfidf)
#print(pred2)
#print(y_test2)
accu_secondary = np.mean(pred2 == y_test2)
print("Accuracy of secondary test dataset is")
print(accu_secondary)

Accuracy of secondary test dataset is
0.7716666666666666


In [131]:
#confusion matrix for primary test dataset
from sklearn import metrics

metrics.confusion_matrix(y_test1, pred)

array([[307,   6],
       [ 76,  61]])

In [132]:
#confusion matrix for secondary test dataset
metrics.confusion_matrix(y_test2, pred2)

array([[923,  16],
       [258,   3]])

In [133]:
from sklearn.metrics import classification_report

print(classification_report(y_test2, pred2))

              precision    recall  f1-score   support

       False       0.78      0.98      0.87       939
        True       0.16      0.01      0.02       261

   micro avg       0.77      0.77      0.77      1200
   macro avg       0.47      0.50      0.45      1200
weighted avg       0.65      0.77      0.69      1200



In [134]:
from sklearn.metrics import classification_report

print(classification_report(y_test1, pred))

              precision    recall  f1-score   support

       False       0.80      0.98      0.88       313
        True       0.91      0.45      0.60       137

   micro avg       0.82      0.82      0.82       450
   macro avg       0.86      0.71      0.74       450
weighted avg       0.83      0.82      0.80       450



In [135]:
#using Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1, random_state = 42)
rf = rf.fit(x_train_tfidf, y_train)

#the accuracy kept on decreasing if number of estimators were increased

In [136]:
#predicting using Random Forest model
pred = rf.predict(x_test1_tfidf)


In [137]:
#performance evaluation with primary test dataset

accu_primary = np.mean(pred == y_test1)
#print(pred)
#print(y_test1)
print("Accuracy of primary test dataset is")
print(accu_primary)

Accuracy of primary test dataset is
0.7533333333333333


In [138]:
#performance evaluation with sceondary dataset

pred2 = rf.predict(x_test2_tfidf)
#print(pred2)
#print(y_test2)
accu_secondary = np.mean(pred2 == y_test2)
print("Accuracy of secondary test dataset is")
print(accu_secondary)

Accuracy of secondary test dataset is
0.7375


In [139]:
from sklearn.metrics import classification_report

print(classification_report(y_test1, pred))

              precision    recall  f1-score   support

       False       0.83      0.82      0.82       313
        True       0.59      0.61      0.60       137

   micro avg       0.75      0.75      0.75       450
   macro avg       0.71      0.71      0.71       450
weighted avg       0.75      0.75      0.75       450



In [140]:
from sklearn.metrics import classification_report

print(classification_report(y_test2, pred2))

              precision    recall  f1-score   support

       False       0.80      0.89      0.84       939
        True       0.32      0.19      0.24       261

   micro avg       0.74      0.74      0.74      1200
   macro avg       0.56      0.54      0.54      1200
weighted avg       0.70      0.74      0.71      1200



In [141]:
from sklearn.linear_model import LogisticRegression

lm = LogisticRegression()
lm = lm.fit(x_train_tfidf, y_train)

/home/akshay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [142]:
#predicting using Random Forest model
pred = lm.predict(x_test1_tfidf)


In [143]:
#performance evaluation with primary test dataset

accu_primary = np.mean(pred == y_test1)
#print(pred)
#print(y_test1)
print("Accuracy of primary test dataset is")
print(accu_primary)

Accuracy of primary test dataset is
0.78


In [144]:
#performance evaluation with sceondary dataset

pred2 = lm.predict(x_test2_tfidf)
#print(pred2)
#print(y_test2)
accu_secondary = np.mean(pred2 == y_test2)
print("Accuracy of secondary test dataset is")
print(accu_secondary)

Accuracy of secondary test dataset is
0.7816666666666666


In [145]:
#Data preprocessing - stop word filtering

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
set(stopwords.words('english'))
stop_words = set(stopwords.words('english')) 

mask_data1['text'] = mask_data1['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))
print(mask_data1["text"])
mask_data2['text'] = mask_data2['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

0                        Language Education Time COVID-19
1                                      COVID-19 Test Kits
2                                      COVID 19 IN PRISON
3                                          Get Waled Home
4       Make pass/fail available Mississippi State Stu...
5       Governor Doug Burgum Issue Mandatory Shelter-i...
6                              Early release due covid 19
7       Medical professionals need protection gear fig...
8       Save Gilroy Recreation - Theater, Aquatics, Pr...
9                            Class 2020 get certification
10      Calling Disability- Inclusive Response Covid-1...
11      Open 2020-2021 School Year Traditionally Shast...
12      Save Enhanced Nursery School Operating Grant!!...
13                Cancel STAAR test 2020/2021 school year
14                            Justice For Dr. Susan Moore
15                     Extend Winter Break Brock Students
16                        Get American crew members home!
17            

In [146]:
print(mask_data2["text"])

0       Putin After Announcing #CovidVaccine #Russian ...
1       Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...
2       4 vaccines Jared bought expected fail Trump re...
3       One day realize CDC Guidelines magically align...
4       Im far lying. Current CDC guidelines wearing m...
5       Russia’s Covid vaccine ‘Sputnik-V’ might relea...
6       I don’t see masks. In fact, child there…and re...
7       I fully vaccinated, unless CDC guidelines chan...
8       ask every single person walks know need wear m...
9       sense touch 40k #CovidVaccine released unregul...
10      Health ministry approval Covid vaccine Russia ...
11      10 Days Onam names | kerala https://t.co/A2nX9...
12      amazing review insights #vaccine experts I hig...
13      Pretty sure people wearing masks right vaccina...
14      These patients slowly tortured fully tortured ...
15      Safety &amp efficacy #CovidVaccine must follow...
16      unveils online platform ease #CovidVaccine #su...
17          On

In [147]:
#from sklearn.metrics import precision_score

#precision_pri1 = precision_score(y_test1, pred)
#print(precision_pri1)
#precision_sec2 = precision_score(y_test2, pred2)
#print(precision_sec2)

In [148]:
#4] Feature Engineering
#adding a new feature - sentiment feature
#from textblob import TextBlob
#from sklearn.preprocessing import MinMaxScaler


#Polarity range lies between -1 to 1. -1 means negative sentiment and 1 means positive sentiment
#def getPolarity(text):
#    polarity = TextBlob(text).sentiment.polarity
#    scaled_mat = (polarity - np.min(polarity) / (np.max(polarity) - np.min(polarity)) * (1 - 0) + 0)
#    scaler = MinMaxScaler()
#    return scaled_mat
#    return scaler.fit_transform(polarity)
#    return TextBlob(text).sentiment.polarity

#mask_data1["sentiment"] = mask_data1["text"].apply(getPolarity)
#mask_data1.head()
#print(mask_data1["sentiment"])

In [149]:
#import matplotlib.pyplot as plt

#plt.plot(mask_data1["sentiment"], mask_data1["label"])

In [150]:
#4] Feature Engineering
#adding a new feature - whether the text contains hashtag

import re

def hashtag_feature(text):
    return int(bool(re.search("#\w+", text)))

mask_data1["hashtag"] = mask_data1["text"].apply(hashtag_feature)

In [151]:
#print(mask_data1["hashtag"])

In [152]:
print(mask_data1.loc[924].at["hashtag"])

1


In [153]:
#adding 2nd feature - length feature
from textblob import TextBlob

mask_data1['word_count'] = mask_data1["text"].apply(lambda x: len(str(x).split(" ")))
#Polarity range lies between -1 to 1. -1 means negative sentiment and 1 means positive sentiment
#def getPolarity(text):
#    return TextBlob(text).sentiment.polarity

#mask_data1["sentiment"] = mask_data1["text"].apply(getPolarity)
mask_data1.head()

,text,label,hashtag,word_count
0,Language Education Time COVID-19,False,0,4
1,COVID-19 Test Kits,False,0,3
2,COVID 19 IN PRISON,False,0,4
3,Get Waled Home,False,0,3
4,Make pass/fail available Mississippi State Stu...,False,0,9


In [154]:
#adding 3rd feature - pattern for detecting the text where COVID is referred
def pattern_detection(text):
    return int(bool(re.search("COVID", text, flags = re.I)))

mask_data1["pattern"] = mask_data1["text"].apply(pattern_detection)

In [155]:
#print(mask_data1["pattern"])

In [156]:
print(mask_data1.loc[888].at["pattern"])

1


In [157]:
#adding 4th feature - pattern for detecting the text where words like close, cancel along with COVID are used which imply actions taken for social distancing
def pattern_detection2(text):
#    return int(bool(re.search("(close|cancel.*|shut|postpone.*)\s[\w\s]+COVID", text, flags = re.I)))
    return int(bool(re.search("(close|cancel.*|shut|postpone.*)\s[\w\s]+COVID", text, flags = re.I)))
mask_data1["pattern2"] = mask_data1["text"].apply(pattern_detection2)

In [158]:
print(mask_data1["pattern2"])

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      1
27      0
28      0
29      0
       ..
1470    0
1471    1
1472    0
1473    0
1474    0
1475    0
1476    0
1477    0
1478    0
1479    0
1480    0
1481    0
1482    0
1483    0
1484    0
1485    0
1486    0
1487    0
1488    0
1489    0
1490    0
1491    0
1492    0
1493    0
1494    0
1495    0
1496    0
1497    0
1498    0
1499    0
Name: pattern2, Length: 1500, dtype: int64


In [159]:
#adding 5th feature - count the number of capital words in a text

def capital_words(text):
    return sum(map(str.isupper, text.split()))

mask_data1["capital_count"] = mask_data1["text"].apply(capital_words)


In [160]:
print(mask_data1.loc[286].at["pattern2"])

1


In [161]:
mask_data1.head()

,text,label,hashtag,word_count,pattern,pattern2,capital_count
0,Language Education Time COVID-19,False,0,4,1,0,1
1,COVID-19 Test Kits,False,0,3,1,0,1
2,COVID 19 IN PRISON,False,0,4,1,0,3
3,Get Waled Home,False,0,3,0,0,0
4,Make pass/fail available Mississippi State Stu...,False,0,9,0,0,0


In [162]:
mask_data1 = mask_data1[["text", "hashtag", "word_count", "pattern", "pattern2","capital_count", "label"]]
mask_data1.head()

,text,hashtag,word_count,pattern,pattern2,capital_count,label
0,Language Education Time COVID-19,0,4,1,0,1,False
1,COVID-19 Test Kits,0,3,1,0,1,False
2,COVID 19 IN PRISON,0,4,1,0,3,False
3,Get Waled Home,0,3,0,0,0,False
4,Make pass/fail available Mississippi State Stu...,0,9,0,0,0,False


In [163]:
#splitting the primary dataset into train and test sets
x_train2, x_test3, y_train2, y_test3 = train_test_split(mask_data1[["text", "hashtag", "word_count", "pattern", "pattern2", "capital_count"]],mask_data1["label"], test_size=0.3, random_state=42 )

In [164]:
x_train2.shape

(1050, 6)

In [165]:
#bag-of-words
count_vect2 = CountVectorizer()
x_train2_counts = count_vect2.fit_transform(x_train2["text"])
x_train2_counts.shape

(1050, 2960)

In [166]:
count_vect2.vocabulary_.get(u'algorithm')

In [167]:
#tf-idf
tfidf = TfidfTransformer()
x_train2_tfidf = tfidf.fit_transform(x_train2_counts)
x_train2_tfidf.shape

(1050, 2960)

In [168]:
#adding 5 new features to train 
#x_train2_final = np.insert(x_train2_tfidf.todense(), x_train2_tfidf.shape[1], x_train2["sentiment"], axis=1)
x_train2_final = np.insert(x_train2_tfidf.todense(), x_train2_tfidf.shape[1], x_train2["hashtag"], axis=1)
x_train2_final = np.insert(x_train2_final, x_train2_final.shape[1], x_train2["word_count"], axis=1)
x_train2_final = np.insert(x_train2_final, x_train2_final.shape[1], x_train2["pattern"], axis=1)
x_train2_final = np.insert(x_train2_final, x_train2_final.shape[1], x_train2["pattern2"], axis=1)
x_train2_final = np.insert(x_train2_final, x_train2_final.shape[1], x_train2["capital_count"], axis=1)
x_train2_final.shape


(1050, 2965)

In [169]:
#tf-idf on test data(primary)
x_test3_counts = count_vect2.transform(x_test3["text"])
x_test3_tfidf = tfidf.transform(x_test3_counts)
x_test3_tfidf.shape

(450, 2960)

In [170]:
#adding 5 new features to test(primary)
#x_test3_final = np.insert(x_test3_tfidf.todense(), x_test3_tfidf.shape[1], x_test3["sentiment"], axis=1)
x_test3_final = np.insert(x_test3_tfidf.todense(), x_test3_tfidf.shape[1], x_test3["hashtag"], axis=1)
x_test3_final = np.insert(x_test3_final, x_test3_final.shape[1], x_test3["word_count"], axis=1)
x_test3_final = np.insert(x_test3_final, x_test3_final.shape[1], x_test3["pattern"], axis=1)
x_test3_final = np.insert(x_test3_final, x_test3_final.shape[1], x_test3["pattern2"], axis=1)
x_test3_final = np.insert(x_test3_final, x_test3_final.shape[1], x_test3["capital_count"], axis=1)
x_test3_final.shape

(450, 2965)

In [171]:
#adding a new feature - sentiment feature to test data
from textblob import TextBlob

#Polarity range lies between -1 to 1. -1 means negative sentiment and 1 means positive sentiment
#def getPolarity(text):
#    return TextBlob(text).sentiment.polarity

#mask_data2["sentiment"] = mask_data2["text"].apply(getPolarity)
mask_data2["hashtag"] = mask_data2["text"].apply(hashtag_feature)
mask_data2['word_count'] = mask_data2["text"].apply(lambda x: len(str(x).split(" ")))
mask_data2["pattern"] = mask_data2["text"].apply(pattern_detection)
mask_data2["pattern2"] = mask_data2["text"].apply(pattern_detection2)
mask_data2["capital_count"] = mask_data2["text"].apply(capital_words)
mask_data2 = mask_data2[["text", "hashtag", "word_count", "pattern", "pattern2","capital_count", "label"]]
mask_data2.head()
#print(mask_data1["sentiment"])

,text,hashtag,word_count,pattern,pattern2,capital_count,label
0,Putin After Announcing #CovidVaccine #Russian ...,1,6,1,0,0,False
1,Courtesy: WA! #WhatsApp #COVID #CovidVaccine h...,1,6,1,0,2,False
2,4 vaccines Jared bought expected fail Trump re...,0,11,0,0,0,False
3,One day realize CDC Guidelines magically align...,0,10,0,0,1,False
4,Im far lying. Current CDC guidelines wearing m...,0,13,0,0,1,True


In [172]:
#using full secondary dataset as test dataset
x_test4 = mask_data2[["text", "hashtag", "word_count", "pattern", "pattern2", "capital_count"]]
y_test4 = mask_data2["label"] == 'True'

In [173]:
#tf-idf on test data(secondary)
x_test4_counts = count_vect2.transform(x_test4["text"])
x_test4_tfidf = tfidf.transform(x_test4_counts)
x_test4_tfidf.shape

(1200, 2960)

In [174]:
#adding 5 new features to test(secondary)
#x_test4_final = np.insert(x_test4_tfidf.todense(), x_test4_tfidf.shape[1], x_test4["sentiment"], axis=1)
x_test4_final = np.insert(x_test4_tfidf.todense(), x_test4_tfidf.shape[1], x_test4["hashtag"], axis=1)
x_test4_final = np.insert(x_test4_final, x_test4_final.shape[1], x_test4["word_count"], axis=1)
x_test4_final = np.insert(x_test4_final, x_test4_final.shape[1], x_test4["pattern"], axis=1)
x_test4_final = np.insert(x_test4_final, x_test4_final.shape[1], x_test4["pattern2"], axis=1)
x_test4_final = np.insert(x_test4_final, x_test4_final.shape[1], x_test4["capital_count"], axis=1)
x_test4_final.shape

(1200, 2965)

In [175]:
#using Naive Bayes 

nb2 = MultinomialNB().fit(x_train2_final, y_train2)
pred3 = nb2.predict(x_test3_final)

#performance evaluation with primary test dataset after adding new features

accu_primary2 = np.mean(pred3 == y_test3)

print("Accuracy of primary test dataset after adding new features is")
print(accu_primary2)

Accuracy of primary test dataset after adding new features is
0.7155555555555555


In [176]:
pred4 = nb2.predict(x_test4_final)
#print(pred2)
#print(y_test2)
accu_secondary2 = np.mean(pred4 == y_test4)
print("Accuracy of secondary test dataset is")
print(accu_secondary2)

Accuracy of secondary test dataset is
0.7825


In [177]:
print(y_test4.dtype)
print(pred4.dtype)

bool
bool


In [178]:
from sklearn.metrics import classification_report

print(classification_report(y_test3, pred3))

              precision    recall  f1-score   support

       False       0.71      1.00      0.83       313
        True       1.00      0.07      0.12       137

   micro avg       0.72      0.72      0.72       450
   macro avg       0.85      0.53      0.48       450
weighted avg       0.80      0.72      0.62       450



In [179]:
from sklearn.metrics import classification_report

print(classification_report(y_test4, pred4))

              precision    recall  f1-score   support

       False       0.78      1.00      0.88       939
        True       0.00      0.00      0.00       261

   micro avg       0.78      0.78      0.78      1200
   macro avg       0.39      0.50      0.44      1200
weighted avg       0.61      0.78      0.69      1200



/home/akshay/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [180]:
#using SVM model on training data
from sklearn.linear_model import SGDClassifier

svm2 = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None).fit(x_train2_final, y_train2)

/home/akshay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [181]:
#predicting using SVM model for primary dataset 
pred3 = svm2.predict(x_test3_final)

In [182]:
#performance evaluation with primary test dataset

accu_primary2 = np.mean(pred3 == y_test3)
#print(pred)
#print(y_test1)
print("Accuracy of primary test dataset is")
print(accu_primary2)

Accuracy of primary test dataset is
0.7933333333333333


In [183]:
#performance evaluation with sceondary dataset

pred4 = svm2.predict(x_test4_final)
#print(pred2)
#print(y_test2)
accu_secondary2 = np.mean(pred4 == y_test4)
print("Accuracy of secondary test dataset is")
print(accu_secondary2)

Accuracy of secondary test dataset is
0.7816666666666666


In [184]:
#from sklearn.metrics import classification_report

#print(classification_report(y_test3, pred3))

In [185]:
#from sklearn.metrics import classification_report

#print(classification_report(y_test4, pred4))

In [186]:
#using Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf2 = RandomForestRegressor(n_estimators = 1, random_state = 42)
rf2 = rf2.fit(x_train2_final, y_train2)

#the accuracy kept on decreasing if number of estimators were increased

In [187]:
#predicting using Random Forest model
pred3 = rf2.predict(x_test3_final)


In [188]:
#performance evaluation with primary test dataset

accu_primary2 = np.mean(pred3 == y_test3)
#print(pred)
#print(y_test1)
print("Accuracy of primary test dataset is")
print(accu_primary2)

Accuracy of primary test dataset is
0.7688888888888888


In [189]:
#performance evaluation with sceondary dataset

pred4 = rf2.predict(x_test4_final)
#print(pred2)
#print(y_test2)
accu_secondary2 = np.mean(pred4 == y_test4)
print("Accuracy of secondary test dataset is")
print(accu_secondary2)

Accuracy of secondary test dataset is
0.71


In [190]:
from sklearn.metrics import classification_report

print(classification_report(y_test3, pred3))

              precision    recall  f1-score   support

       False       0.81      0.87      0.84       313
        True       0.64      0.55      0.59       137

   micro avg       0.77      0.77      0.77       450
   macro avg       0.73      0.71      0.71       450
weighted avg       0.76      0.77      0.76       450



In [191]:
from sklearn.metrics import classification_report

print(classification_report(y_test4, pred4))

              precision    recall  f1-score   support

       False       0.79      0.86      0.82       939
        True       0.24      0.15      0.19       261

   micro avg       0.71      0.71      0.71      1200
   macro avg       0.51      0.51      0.51      1200
weighted avg       0.67      0.71      0.69      1200



In [192]:
from sklearn.linear_model import LogisticRegression

lm2 = LogisticRegression()
lm2 = lm2.fit(x_train2_final, y_train2)

/home/akshay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [193]:
#predicting using Logistic Regression model
pred3 = lm2.predict(x_test3_final)


In [194]:
#performance evaluation with primary test dataset

accu_primary2 = np.mean(pred3 == y_test3)
#print(pred)
#print(y_test1)
print("Accuracy of primary test dataset is")
print(accu_primary2)

Accuracy of primary test dataset is
0.78


In [195]:
#performance evaluation with sceondary dataset

pred4 = lm2.predict(x_test4_final)
#print(pred2)
#print(y_test2)
accu_secondary2 = np.mean(pred4 == y_test4)
print("Accuracy of secondary test dataset is")
print(accu_secondary2)

Accuracy of secondary test dataset is
0.7816666666666666


Above, I have implemented 4 classification models both before feature engineering and after feature engineering.

Before feature engineering:
Accuracy of primary test dataset
Naive Bayes - 75.55%
SVM - 81.77%
Random Forest - 75.33%
Logistic Regression - 78%

Accuracy of secondary test dataset
Naive Bayes - 78.25%
SVM - 77.16%
Random Forest - 73.75%
Logistic Regression - 78.16%

Accuracy of primary test dataset
Naive Bayes - 71.55%
SVM - 79.33%
Random Forest - 76.88%
Logistic Regression - 78%

Accuracy of secondary test dataset
Naive Bayes - 78.25%
SVM - 78.16%
Random Forest - 71%
Logistic Regression - 78.16%